# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Colon and Rectal Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [3]:
trait_subdir = "TCGA_Colon_and_Rectal_Cancer_(COADREAD)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.COADREAD.sampleMap_COADREAD_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.COADREAD.sampleMap_HiSeqV2_PANCAN.gz')

In [4]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [5]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [6]:
clinical_data_cols[:10]

['AWG_MLH1_silencing',
 'AWG_cancer_type_Oct62011',
 'CDE_ID_3226963',
 'CIMP',
 'MSI_updated_Oct62011',
 '_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_COADREAD',
 '_PANCAN_DNAMethyl_PANCAN']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [7]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AWG_MLH1_silencing', 'AWG_cancer_type_Oct62011', 'CDE_ID_3226963', 'CIMP', 'MSI_updated_Oct62011', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_COADREAD', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_

In [8]:
# candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'year_of_initial_pathologic_diagnosis']

# candidate_gender_cols = ['gender']


candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [9]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [61.0, 67.0, 42.0, 74.0, nan],
 'days_to_birth': [-22379.0, -24523.0, -15494.0, -27095.0, nan],
 'days_to_collection': [35.0, nan, 107.0, 1778.0, nan],
 'days_to_death': [nan, nan, nan, '49', nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, nan],
 'days_to_last_followup': ['475', '146', '385', nan, nan],
 'days_to_new_tumor_event_additional_surgery_procedure': [nan,
  nan,
  nan,
  nan,
  nan],
 'days_to_new_tumor_event_after_initial_treatment': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2013.0, 2013.0, 2013.0, 2009.0, nan]}

In [10]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [11]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', nan]}

In [12]:
gender_col = 'gender'

In [13]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [14]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [15]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Colon and Rectal Cancer,Age,Gender,UBE2Q1,RNF14,SERPINB11,RNF17,RNF10,RNF11,RNF13,...,SLC7A10,SLC7A11,FOLR1,TULP2,NPY5R,GOLGA6D,SERPINH1,NFIX,SELP,ADAMTS19
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3L-AA1B-01,1,61.0,0.0,0.615782,3.85144,-0.640695,-9.802658,0.615399,-0.424735,-2.504718,...,-0.791986,6.723118,-1.061697,-0.748878,-0.034117,-1.06467,-0.262197,3.141164,-3.250033,-0.199484
TCGA-4N-A93T-01,1,67.0,1.0,0.908282,4.08014,-0.640695,-9.802658,0.635099,-1.425235,-1.654018,...,1.462614,6.109118,-7.163097,-0.748878,3.199883,-1.06467,-0.406497,3.286964,-7.381133,-3.482084
TCGA-4T-AA8H-01,1,42.0,0.0,0.252882,3.50444,-0.640695,-9.802658,0.816899,-0.679735,-2.327818,...,-2.090786,6.204618,-1.205497,-0.748878,1.335083,-1.06467,-1.116797,2.812764,-6.648433,-5.035584
TCGA-5M-AAT4-01,1,74.0,1.0,0.946082,3.29594,-0.640695,-9.286358,0.339999,-1.385135,-2.217018,...,-0.060386,7.695418,-4.704097,-0.232578,-1.070817,-1.06467,-0.230397,3.358264,-5.788033,-3.379584
TCGA-5M-AAT6-01,1,40.0,0.0,0.558082,3.41174,0.087205,-9.802658,0.552499,-0.555435,-2.594018,...,-1.340186,8.269918,-5.668797,0.460622,-1.587117,-1.06467,1.261403,2.274064,-1.230233,-1.284084


In [16]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 430 samples.


In [17]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Colon and Rectal Cancer', the least common label is '0' with 51 occurrences. This represents 11.86% of the dataset.
The distribution of the feature 'Colon and Rectal Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 67.0
  75%: 75.0
Min: 31.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 199 occurrences. This represents 46.28% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [18]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [19]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: C:/Users/arthu/OneDrive/Documents/GitHub/output1\Arthur\Colon-and-Rectal-Cancer\cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [19]:
dataset = 'GEO'
trait_subdir = "Colon-and-Rectal-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE103512',
 'GSE108166',
 'GSE10982',
 'GSE110785',
 'GSE115513',
 'GSE12225',
 'GSE15781',
 'GSE25229',
 'GSE39958',
 'GSE46517',
 'GSE46862',
 'GSE56699']

Repeat the below steps for all the accession numbers

In [20]:
cohort = accession_num = "GSE103512"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Colon-and-Rectal-Cancer\\GSE103512\\GSE103512_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Colon-and-Rectal-Cancer\\GSE103512\\GSE103512_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [21]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Gene expression profiles of breast, colorectal, prostate, and non-small cell lung cancer"
!Series_summary	"Gene expression profiles from 280 formalin-fixed and paraffin embedded normal and tumor samples of four cancer types"
!Series_overall_design	"65 breast cancer with 10 matched normal; 57 colorectal cancer with 12 matched normal; 60 non-small cell lung cancer with 9 matched normal; 60 prostate cancer with 7 matched normal"


In [22]:
clinical_data.head()

,!Sample_geo_accession,GSM2772660,GSM2772661,GSM2772662,GSM2772663,GSM2772664,GSM2772665,GSM2772666,GSM2772667,GSM2772668,...,GSM2772930,GSM2772931,GSM2772932,GSM2772933,GSM2772934,GSM2772935,GSM2772936,GSM2772937,GSM2772938,GSM2772939
0,!Sample_characteristics_ch1,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,cancer type: BC,...,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA,cancer type: PCA
1,!Sample_characteristics_ch1,normal: no,normal: no,normal: no,normal: no,normal: no,normal: no,normal: no,normal: no,normal: no,...,normal: no,normal: no,normal: no,normal: yes,normal: yes,normal: yes,normal: yes,normal: yes,normal: yes,normal: yes
2,!Sample_characteristics_ch1,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,batch i/ii: I,...,histology: na,histology: na,histology: na,batch i/ii: II,batch i/ii: II,batch i/ii: II,batch i/ii: II,batch i/ii: II,batch i/ii: II,batch i/ii: II
3,!Sample_characteristics_ch1,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na,...,age: na,age: na,age: na,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na,histology: na
4,!Sample_characteristics_ch1,age: na,age: na,age: na,age: na,age: na,age: na,age: na,age: na,age: na,...,gender: na,gender: na,gender: na,age: na,age: na,age: na,age: na,age: na,age: na,age: na


In [23]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cancer type: BC',
  'cancer type: CRC',
  'cancer type: NSCLC',
  'cancer type: PCA'],
 1: ['normal: no', 'normal: yes'],
 2: ['batch i/ii: I',
  'histology: ductal carcinoma',
  'histology: na',
  'batch i/ii: II',
  'histology: adenocarcinoma with acinar type',
  'histology: adenocarcinoma'],
 3: ['histology: na',
  'histology: ductal carcinoma',
  'histology: lobular carcinoma',
  'age: 63',
  'histology: carcinoma',
  'histology: mucinous adenocarcinoma',
  'age: na',
  'age: 38',
  'histology: adenocarcinoma',
  'histology: partially mucinous adenocarcinoma',
  'histology: squamous cell carcinoma',
  'histology: adenocarcinoma with mixed subtypes',
  'age: 64',
  'age: 58',
  'age: 62',
  'age: 71',
  'age: 73',
  'age: 49',
  'age: 55',
  'age: 50',
  'age: 59',
  'age: 56',
  'age: 65',
  'age: 66',
  'age: 52',
  'age: 67',
  'age: 70',
  'age: 76',
  'age: 53',
  'age: 68'],
 4: ['age: na',
  'age: 79',
  'age: 82',
  'age: 58',
  'age: 66',
  'age: 46',
  'age: 63',
  '

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [24]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Colon and Rectal Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Colon and Rectal Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is re

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [25]:
is_gene_availabe = True
trait_row = 0
age_row = 3 and 4
gender_row = 5

trait_type = 'binary'

In [26]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [27]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'primary diagnosis: PSORIASIS':
#         return 1  # Epilepsy present
#     else:
#         return 0  # Epilepsy not present

# def convert_age(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age: '):
#         try:
#             return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: Female':
#         return 0
#     elif gender_string == 'Sex: Male':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender

def convert_trait(value):
    if value == 'cancer type: CRC':
        return True
    elif value == 'cancer type: BC':
        return False
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None



def convert_gender(value):
    if value == 'gender: m':
        return 'male'
    elif value == 'gender: f':
        return 'female'
    else:
        return None




In [28]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM2772660,GSM2772661,GSM2772662,GSM2772663,GSM2772664,GSM2772665,GSM2772666,GSM2772667,GSM2772668,GSM2772669,...,GSM2772930,GSM2772931,GSM2772932,GSM2772933,GSM2772934,GSM2772935,GSM2772936,GSM2772937,GSM2772938,GSM2772939
Colon and Rectal Cancer,False,False,False,False,False,False,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
Age,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Gender,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Genetic data preprocessing and final filtering

In [29]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM2772660,GSM2772661,GSM2772662,GSM2772663,GSM2772664,GSM2772665,GSM2772666,GSM2772667,GSM2772668,GSM2772669,...,GSM2772930,GSM2772931,GSM2772932,GSM2772933,GSM2772934,GSM2772935,GSM2772936,GSM2772937,GSM2772938,GSM2772939
ID,,,,,,,,,,,,,,,,,,,,,
1007_PM_s_at,8.710718,8.874406,9.157600,7.710366,7.889133,9.187891,8.502790,7.985991,9.288803,8.037423,...,7.591758,8.417337,8.773584,6.014932,8.118485,7.750079,7.864324,7.541111,8.390298,7.755427
1053_PM_at,4.646897,5.946730,5.769209,4.512610,6.000142,4.529334,5.326246,5.772236,5.757419,5.547636,...,4.757886,4.622830,4.427917,4.139551,4.374756,4.173053,4.259325,4.522224,4.355016,4.367581
117_PM_at,6.850312,7.420204,7.277934,7.036745,7.085630,7.001320,7.226530,7.395984,6.985698,7.915826,...,7.579409,6.720147,6.928376,7.491676,6.353371,6.424754,6.478735,6.865915,6.765574,7.624744
121_PM_at,5.699862,5.955848,5.819856,5.992074,5.965098,5.978569,5.975539,6.133506,5.623385,5.919976,...,6.244688,6.067174,5.778329,6.482538,5.982061,6.619358,7.989656,6.025906,6.136103,6.106748
1255_PM_g_at,3.344434,3.252921,3.121310,3.304214,3.206685,3.123327,3.070930,3.161671,3.308474,3.214317,...,3.321744,3.119197,3.081535,3.376316,3.253850,3.114041,3.135294,3.328501,3.282186,3.310275


In [30]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_PM_s_at',
 '1053_PM_at',
 '117_PM_at',
 '121_PM_at',
 '1255_PM_g_at',
 '1294_PM_at',
 '1316_PM_at',
 '1320_PM_at',
 '1405_PM_i_at',
 '1431_PM_at',
 '1438_PM_at',
 '1487_PM_at',
 '1494_PM_f_at',
 '1552256_PM_a_at',
 '1552257_PM_a_at',
 '1552258_PM_at',
 '1552261_PM_at',
 '1552263_PM_at',
 '1552264_PM_a_at',
 '1552266_PM_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [31]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at', '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at', '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at', '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at', '1552264_PM_a_at', '1552266_PM_at']\n"

If not required, jump directly to the gene normalization step

In [33]:
requires_gene_mapping = True

In [34]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITI

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [35]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affym

In [36]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [37]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [38]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [39]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 110 samples.


In [40]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Colon and Rectal Cancer', the least common label is 'False' with 55 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Colon and Rectal Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 68.5
  75%: 76.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is 'male' with 26 occurrences. This represents 23.64% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [41]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [42]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE123088'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE123088,True,True,False,True,True,192,
1,GSE123086,True,True,False,True,True,154,


In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)